# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('hack_prj').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

dataset = spark.read.csv('hack_data.csv', inferSchema=True, header=True)

In [4]:
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [9]:
dataset.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [10]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [11]:
dataset.select('Location').show()

+--------------------+
|            Location|
+--------------------+
|            Slovenia|
|British Virgin Is...|
|             Tokelau|
|             Bolivia|
|                Iraq|
|    Marshall Islands|
|             Georgia|
|         Timor-Leste|
|Palestinian Terri...|
|          Bangladesh|
|Northern Mariana ...|
|            Zimbabwe|
|         Isle of Man|
|Sao Tome and Prin...|
|              Greece|
|     Solomon Islands|
|       Guinea-Bissau|
|        Burkina Faso|
|            Mongolia|
|             Nigeria|
+--------------------+
only showing top 20 rows



In [15]:
temp = dataset.groupBy('Location').count()
temp.orderBy(temp['count'].desc()).show()

+--------------------+-----+
|            Location|count|
+--------------------+-----+
|United States Vir...|    6|
|          Mauritania|    5|
|      Czech Republic|    5|
|       Guinea-Bissau|    5|
| Antigua and Barbuda|    4|
|    Saint Barthelemy|    4|
|              Tuvalu|    4|
|             Grenada|    4|
|Palestinian Terri...|    4|
|             Ukraine|    4|
|             Reunion|    4|
|           Sri Lanka|    4|
| Trinidad and Tobago|    4|
|         Netherlands|    4|
|Northern Mariana ...|    3|
|Netherlands Antilles|    3|
|French Southern T...|    3|
|                Iraq|    3|
|              Bhutan|    3|
|         Isle of Man|    3|
+--------------------+-----+
only showing top 20 rows



In [16]:
dataset.select('Location').distinct().count()

181

In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [20]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],
                           outputCol='feature')

In [21]:
final_data = assembler.transform(dataset)

In [22]:
from pyspark.ml.feature import StandardScaler

In [24]:
scaler = StandardScaler(inputCol='feature', outputCol='scaled_features')

In [27]:
scaler_model= scaler.fit(final_data)

final_data = scaler_model.transform(final_data)

In [29]:
final_data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'feature',
 'scaled_features']

### Train and evaluate

In [46]:
kmeans1 = KMeans(featuresCol='scaled_features', k=2)
kmeans2 = KMeans(featuresCol='scaled_features', k=3)

model1 = kmeans1.fit(final_data)
model2 = kmeans2.fit(final_data)

In [47]:
wssse = model1.computeCost(final_data)
print(f'WSSSE: {wssse}')

WSSSE: 601.7707512676716


In [48]:
model1.clusterCenters()

[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
        5.26676612])]

In [49]:
preds1 = model1.transform(final_data)
preds2 = model2.transform(final_data)

In [50]:
preds1.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'feature',
 'scaled_features',
 'prediction']

In [51]:
preds1.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [52]:
preds2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+

